In [1]:
%reload_ext dotenv
%dotenv

In [2]:
%env LOGLEVEL=INFO

%load_ext autoreload
%autoreload 2

env: LOGLEVEL=INFO


In [3]:
import os, logging
import json

print(os.getenv("LOGLEVEL"))
# logging.basicConfig(level=os.environ.get("LOGLEVEL", "WARNING").upper())
log = logging.getLogger("portfolio-transactions")
log.setLevel(os.environ.get("LOGLEVEL", "WARNING").upper())

INFO


In [4]:
import sys
print(sys.path)

['/home/harjeet/playground/KiteConnect/options-charts/notebooks', '/home/harjeet/.pyenv/versions/3.8.3/lib/python38.zip', '/home/harjeet/.pyenv/versions/3.8.3/lib/python3.8', '/home/harjeet/.pyenv/versions/3.8.3/lib/python3.8/lib-dynload', '', '/home/harjeet/.virtualenvs/kiteconnect/lib/python3.8/site-packages', '/home/harjeet/playground/KiteConnect/options-charts/src', '/home/harjeet/.virtualenvs/kiteconnect/lib/python3.8/site-packages/IPython/extensions', '/home/harjeet/.ipython']


## References

1. [Kite Connect 3 / API documentation](https://kite.trade/docs/connect/v3/)
    - [The Kite Connect API Python client - v3](https://github.com/zerodhatech/pykiteconnect/)
    - [developers.kite.trade console](https://developers.kite.trade/apps)
2. [zerodhatech on github](https://github.com/zerodhatech)   
1. [Calculating Implied volatility](https://medium.com/pythoptions/calculate-implied-volatility-or-any-options-greek-in-just-3-lines-in-python-f251e77d1e24)
    - [mibian](http://code.mibian.net/) library
2. Google search 
    - [candlestick charts with jscharts](https://www.google.com/search?q=candlestick+charts+with+jscharts&oq=candlestick+charts+with+jscharts&aqs=chrome..69i57.7466j0j1&sourceid=chrome&ie=UTF-8)
    - [create dynamic Candlestick Charts](https://www.google.com/search?q=create+dynamic+Candlestick+Charts&oq=create+dynamic+Candlestick+Charts&aqs=chrome..69i57.353j0j1&sourceid=chrome&ie=UTF-8)
3. Building candlestick charts dynamically
    - [Ploting candlestick](https://plotly.com/python/candlestick-charts/)
    - [Hands-On Guide To Using YFinance API In Python](https://analyticsindiamag.com/hands-on-guide-to-using-yfinance-api-in-python/)
    - [techanjs.org](http://techanjs.org/) - A visual, stock charting (Candlestick, OHLC, indicators) and technical analysis library built on D3.
    - [anychart](https://www.anychart.com/products/anystock/overview/)
    - [jscharting - finance-stock/candlestick](https://jscharting.com/examples/chart-types/finance-stock/candlestick/)
    - [Google Charts - Candlestick Charts](https://developers.google.com/chart/interactive/docs/gallery/candlestickchart) - [tutorial](https://www.wikitechy.com/tutorials/google-charts/google-charts-candlestick-charts)
    - [canvasjs JavaScript Candlestick Charts & Graphs](https://canvasjs.com/javascript-candlestick-chart/)
    - [chart.js](https://www.chartjs.org/samples/latest/)
        - [scale - timeseries](https://www.chartjs.org/samples/latest/scales/time/financial.html)
        - [udemy course](https://www.udemy.com/course/chart-js/)
    - Highcharts
        - https://jsfiddle.net/BlackLabel/uparh9xo/
        - [candlestick](https://www.highcharts.com/demo/stock/candlestick)
        - [dynamic updates](https://www.highcharts.com/stock/demo/dynamic-update)
    - [TradingView Financial HTML5 Charts](https://www.tradingview.com/HTML5-stock-forex-bitcoin-charting-library/) 
        - [charting examples](https://github.com/tradingview/charting-library-examples)
        - [lightweight-charts](https://github.com/tradingview/lightweight-charts)
        - [TradingView JS API Integration Tutorial](https://medium.com/@jonchurch/tradingview-js-api-integration-tutorial-introduction-5e4809d9ef36)
        - [TradingView Charting Library JS API Setup for Crypto](https://medium.com/@jonchurch/tradingview-charting-library-js-api-setup-for-crypto-part-1-57e37f5b3d5a)
        - [TradingView Charting Library Tutorial](https://github.com/tradingview/charting-library-tutorial)
        - [TradingView Charting Library Integration Examples](https://github.com/tradingview/charting-library-examples)
        - [Awesome TradingView](https://github.com/tradingview/awesome-tradingview)
3. [similartech.com charting](https://www.similartech.com/categories/charting)
    - [similartech.com/compare/highcharts-vs-tradingview-charting-library](https://www.similartech.com/compare/highcharts-vs-tradingview-charting-library)
4. Tick data transformations
    - [Forming Candles in python using Zerodha Websocket](http://ezeetrading.in/Articles/Candles_formation_from_tick_data_zerodha.html)
    - [youtube - Converting WebSocket Streaming Data into OHLC/Candle(https://github.com/tradingview/charting-library-examples)1m,3m,5m,15m,30m,60m,1D) Data](https://www.youtube.com/watch?v=LUvb6XwEc1U)
    - https://gist.github.com/oldmonkABA/f73f5d67bcf85316ac0470919bf54e17
    - https://kite.trade/forum/discussion/6230/program-for-converting-tick-data-into-candlestick-data#latest
5. [jsonformatter.org](https://jsonformatter.org/)    
    
**script3 takes the list of dicts per instrument, one at a time and converts it into ohlc as follows:**
source: https://kite.trade/forum/discussion/2604/convert-ticks-to-candle#latest

```python
import pandas.
pd2 = pandas.DataFrame( instrument1 )
ticks = pd2.ix[:, [ 'ltp' , 'volume' ] ]
priceOHLCV = ticks.ltp.resample( '1min' ).ohlc()
candledata = priceOHLCV.to_csv() # converts the pandas dataframe candle data to csv format written to db which can be easily processed further.  
```

### Highcharts

simplest way to make your chart is to use series.removePoint option before you are using series.addPoint.
http://api.highcharts.com/highstock#Series.addPoint
http://api.highcharts.com/highstock#Series.removePoint

Here you can see simple example how it can work:
http://jsfiddle.net/turgya55/3/

You can also use Series.setData, but I don't think that this will be better idea:
http://api.highcharts.com/highstock#Series.setData



### cronjob

```cron
45 03 * * 1-5 /home/harjeet/options-charts/bin/option_charts.sh
```

In [5]:
from kiteconnect import KiteConnect

In [6]:
API_KEY      = os.getenv("API_KEY")
API_SECRET   = os.getenv("API_SECRET")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

## Getting Access Token

Source: https://kite.zerodha.com/connect/login?v=3&api_key=xxx

In [ ]:
!echo "https://kite.zerodha.com/connect/login?v=3&api_key=$API_KEY"

In [1]:
!echo $ACCESS_TOKEN

iWj09unO1Zvf3EPtrdFR6aFmgU1lzd7C


In [8]:
request_token = "PpIHYsuX9NFUpgCntdgc0OPUhexhTnoW"

def get_access_token():
    kite = KiteConnect(api_key=os.getenv("API_KEY")) # api_key is in above url
    data = kite.generate_session(request_token, os.getenv("API_SECRET"))
    access_token = data["access_token"]
    return access_token

# get_access_token()

'yrXnBeuNUBI1QZptPd8aqMuhxl162JOG'

```sh
curl "https://api.kite.trade/instruments" \
    -H "X-Kite-Version: 3" \
  -H "Authorization: token $API_KEY:$ACCESS_TOKEN"
```  

In [ ]:
!curl "https://api.kite.trade/instruments" \
    -H "X-Kite-Version: 3" \
  -H "Authorization: token $API_KEY:$ACCESS_TOKEN"

In [3]:
!pip list

Package             Version                       Location
------------------- ----------------------------- -------------------------------------------------------
amqp                5.0.1
argon2-cffi         20.1.0
astroid             2.4.2
async-generator     1.10
attrs               20.2.0
autobahn            20.7.1
Automat             20.2.0
backcall            0.2.0
billiard            3.6.3.0
bleach              3.2.1
celery              5.0.2
certifi             2020.6.20
cffi                1.14.3
chardet             3.0.4
click               7.1.2
click-default-group 1.2.2
click-didyoumean    0.0.3
click-repl          0.1.6
constantly          15.1.0
cryptography        3.2.1
decorator           4.4.2
defusedxml          0.6.0
entrypoints         0.3
enum34              1.1.10
hyperlink           20.0.1
idna                2.10
incremental         17.5.0
ipykernel           5.3.4
ipython             7.19.0
ipython-genutils    0.2.0
isort               5.6.4
jedi             

## Get Tick

In [10]:
instruments = [12219650, 12219906]

In [11]:
import logging
from kiteconnect import KiteTicker

logging.basicConfig(level=logging.DEBUG)

# Initialise
# kws = KiteTicker("your_api_key", "your_access_token")
kws = KiteTicker(API_KEY, ACCESS_TOKEN)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    
    # on json.dumps
    # TypeError: Object of type datetime is not JSON serializable formatting issue on datetime object 
    #     http://127.0.0.1:7777/?token=473bbc2c11c1b9b0865b35b31c0ba704c151a06b833abce7
    logging.debug("Ticks: {}".format(json.dumps(ticks, indent=4, default=str)))

def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
#     ws.subscribe([738561, 5633])
    ws.subscribe(instruments)

    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_FULL, [738561])
    ws.set_mode(ws.MODE_FULL, instruments)

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close


```json
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 12219650, 'last_price': 212.9, 'last_quantity': 75, 'average_price': 227.14, 'volume': 1348650, 'buy_quantity': 61500, 'sell_quantity': 67575, 'ohlc': {'open': 265.1, 'high': 268.3, 'low': 205.05, 'close': 264.15}, 'change': -19.401855006625016}, {'tradable': True, 'mode': 'quote', 'instrument_token': 12219906, 'last_price': 272.65, 'last_quantity': 75, 'average_price': 267.43, 'volume': 92700, 'buy_quantity': 58350, 'sell_quantity': 45750, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 3.866666666666658}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 12219906, 'last_price': 272.65, 'last_quantity': 75, 'average_price': 267.43, 'volume': 92700, 'buy_quantity': 57000, 'sell_quantity': 47025, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 3.866666666666658}]
```


**ws.MODE_FULL**

```json
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 12219650, 'last_price': 210.55, 'last_quantity': 75, 'average_price': 227.19, 'volume': 1344000, 'buy_quantity': 61650, 'sell_quantity': 64725, 'ohlc': {'open': 265.1, 'high': 268.3, 'low': 205.05, 'close': 264.15}, 'change': -20.291501041075136, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 46), 'oi': 834150, 'oi_day_high': 834150, 'oi_day_low': 735225, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 50), 'depth': {'buy': [{'quantity': 150, 'price': 210.0, 'orders': 2}, {'quantity': 75, 'price': 209.9, 'orders': 1}, {'quantity': 75, 'price': 209.85, 'orders': 1}, {'quantity': 75, 'price': 209.8, 'orders': 1}, {'quantity': 75, 'price': 209.7, 'orders': 1}], 'sell': [{'quantity': 75, 'price': 210.45, 'orders': 1}, {'quantity': 75, 'price': 210.5, 'orders': 1}, {'quantity': 150, 'price': 210.6, 'orders': 2}, {'quantity': 300, 'price': 210.65, 'orders': 2}, {'quantity': 150, 'price': 210.7, 'orders': 2}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 12219906, 'last_price': 277.0, 'last_quantity': 75, 'average_price': 267.28, 'volume': 90825, 'buy_quantity': 53925, 'sell_quantity': 44625, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 5.523809523809524, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 49), 'oi': 54525, 'oi_day_high': 56250, 'oi_day_low': 50100, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 51), 'depth': {'buy': [{'quantity': 75, 'price': 277.0, 'orders': 1}, {'quantity': 150, 'price': 276.05, 'orders': 1}, {'quantity': 225, 'price': 276.0, 'orders': 1}, {'quantity': 75, 'price': 275.9, 'orders': 1}, {'quantity': 600, 'price': 275.8, 'orders': 1}], 'sell': [{'quantity': 825, 'price': 277.9, 'orders': 2}, {'quantity': 75, 'price': 278.0, 'orders': 1}, {'quantity': 150, 'price': 278.1, 'orders': 1}, {'quantity': 1125, 'price': 278.15, 'orders': 1}, {'quantity': 1125, 'price': 278.25, 'orders': 1}]}}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 12219906, 'last_price': 277.0, 'last_quantity': 75, 'average_price': 267.28, 'volume': 90825, 'buy_quantity': 53925, 'sell_quantity': 44625, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 5.523809523809524, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 49), 'oi': 54525, 'oi_day_high': 56250, 'oi_day_low': 50100, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 51), 'depth': {'buy': [{'quantity': 75, 'price': 277.0, 'orders': 1}, {'quantity': 150, 'price': 276.05, 'orders': 1}, {'quantity': 225, 'price': 276.0, 'orders': 1}, {'quantity': 75, 'price': 275.9, 'orders': 1}, {'quantity': 600, 'price': 275.8, 'orders': 1}], 'sell': [{'quantity': 825, 'price': 277.9, 'orders': 2}, {'quantity': 75, 'price': 278.0, 'orders': 1}, {'quantity': 150, 'price': 278.1, 'orders': 1}, {'quantity': 1125, 'price': 278.15, 'orders': 1}, {'quantity': 1125, 'price': 278.25, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 12219650, 'last_price': 210.55, 'last_quantity': 75, 'average_price': 227.19, 'volume': 1344000, 'buy_quantity': 60150, 'sell_quantity': 64800, 'ohlc': {'open': 265.1, 'high': 268.3, 'low': 205.05, 'close': 264.15}, 'change': -20.291501041075136, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 46), 'oi': 834150, 'oi_day_high': 834150, 'oi_day_low': 735225, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 51), 'depth': {'buy': [{'quantity': 150, 'price': 210.0, 'orders': 2}, {'quantity': 75, 'price': 209.9, 'orders': 1}, {'quantity': 75, 'price': 209.85, 'orders': 1}, {'quantity': 75, 'price': 209.8, 'orders': 1}, {'quantity': 75, 'price': 209.7, 'orders': 1}], 'sell': [{'quantity': 75, 'price': 210.3, 'orders': 1}, {'quantity': 75, 'price': 210.45, 'orders': 1}, {'quantity': 75, 'price': 210.5, 'orders': 1}, {'quantity': 150, 'price': 210.6, 'orders': 2}, {'quantity': 300, 'price': 210.65, 'orders': 2}]}}]
```


In [12]:
kws.connect()

DEBUG:root:Ticks: [
    {
        "tradable": true,
        "mode": "full",
        "instrument_token": 12219906,
        "last_price": 262.9,
        "last_quantity": 75,
        "average_price": 260.84,
        "volume": 147525,
        "buy_quantity": 49950,
        "sell_quantity": 39225,
        "ohlc": {
            "open": 269.6,
            "high": 294.15,
            "low": 212.95,
            "close": 262.5
        },
        "change": 0.1523809523809437,
        "last_trade_time": "2020-11-04 14:12:14",
        "oi": 52725,
        "oi_day_high": 56250,
        "oi_day_low": 50100,
        "timestamp": "2020-11-04 14:12:45",
        "depth": {
            "buy": [
                {
                    "quantity": 75,
                    "price": 261.4,
                    "orders": 1
                },
                {
                    "quantity": 1200,
                    "price": 261.35,
                    "orders": 2
                },
                {
             

## Sqlite3 insertion

In [25]:
import sqlite3
from datetime import datetime

In [26]:
db = sqlite3.connect(os.getenv("DB_FILE"))
DB_FILE = os.getenv("DB_FILE")

In [30]:
c = db.cursor()
# for tick in ticks:
c.execute(f"insert into ticks values ('{datetime.now()}', 12345, 23.5)")
	# c.execute(insert_tick_statement, {
	# 	"date": datetime.now(),
	# 	"token": tick["instrument_token"],
	# 	"price": tick["last_price"]})

# logging.info("Inserting ticks to db : {}".format(json.dumps(ticks)))

try:
	db.commit()
except Exception:
	db.rollback()
	logging.exception("Couldn't write ticks to db: ")